In [2]:
import main_module as m
import zoho_filter_module as fil
import pandas as pd
import re
import numpy as np

%load_ext autoreload
%autoreload 2

cols = ['Record Id',	
'Created Time',
'Created Date',	
'First Name',	
'Last Name',	
'Email',	
'PHONE',	
'Mobile',	
'Birthdate',	
'Year of Birth',
'Street',	
'Street2',	
'City',	
'State',	
'Country',	
'Nationality',
'Booking Status', 
'BookRef',
'Departure Date',	
'Arrival Date',	
'Card Type',	
'Marital Status',	
'Lead Brand',	
'Lead Sub-Brand',	
'Lead Source',	
'Lead Locations',	
'Lead Source Description',	
'Lead Regions',
'Lead Status',
'Email Opt Out',	
'Opt In']

In [3]:
path = r'C:\Users\fajar.fatoni\Documents\Python\Data\Leads_2025_03_07.csv'
zoho_df = pd.read_csv(path, dtype='string')
# zoho_df = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

zoho_df.reset_index(drop=True, inplace=True)
count_filter_awal = zoho_df.shape[0]

# mengisi kolom <NA> dengan value string 'empty' supaya mudah mengecek data member dan booking dengan isin
zoho_df.fillna('empty', inplace=True)

zoho_df['Mobile'] = 'empty'
# zoho_df['Phone'] = 'empty'
zoho_df

,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,...,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?,Mobile
0,zcrm_3584690000001379021,2019-02-03 08:42:45,Keli,Parkes,keli@keliaroha.com,1976-05-14,+61413422227,"17 Fishnourne rd, Allambie heights",Sydney,Australia,...,Other,Web Enquiry,Karma Beach VIP membership sign up 2023,empty,"Other - Web Enquiry, Karma Club - Membership,...",https://karmagroup.com/karma-beach-vip/,2024000100260004,2025-01-26,false,empty
1,zcrm_3584690000002932135,2019-02-28 03:24:54,Almas Hussain,Rizvi,almas_h@yahoo.com,empty,+919886790473,empty,empty,India,...,Other,Facebook Ads,Karma Beach VIP membership sign up 2023,General,Instagram,https://karmagroup.com/karma-beach-vip/,2024000100090124,2025-01-09,false,empty
2,zcrm_3584690000005944396,2019-04-02 02:14:44,Caryl,Costello,carylcostello@yahoo.com.au,1940-01-01,+61417074510,"131 College Road, Karana Downs",Brisbane,Australia,...,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Jimbaran,"Other - Past Guests, Other - Web Chat, Karma C...",https://karmagroup.com/karma-beach-vip/,2024000800300003,2025-08-30,false,empty
3,zcrm_3584690000005945085,2019-04-02 02:14:48,Dawn,Poole,chrisdawnpoole@gmail.com,empty,+61417854221,empty,empty,Australia,...,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Bavaria,"Other - Past Guests, Other - Other",https://karmagroup.com/karma-beach-vip/,2024000500230006,2025-05-23,false,empty
4,zcrm_3584690000005948267,2019-04-02 02:15:06,Elycia,Martins,elycia.martins@gmail.com,empty,+61403924069,empty,empty,Australia,...,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Rottnest,Other - Past Guests,https://karmagroup.com/karma-beach-vip/,2024000900090001,2025-09-09,false,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10392,zcrm_3584690000526811033,2025-03-07 09:13:36,Ekaterina,Terekhina,katyess@icloud.com,empty,+79136194604,empty,empty,Russia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300070002,2026-03-07,false,empty
10393,zcrm_3584690000526812042,2025-03-07 10:49:34,Juliia,Parshenkova,ylashka_1@mail.ru,empty,+79060677484,empty,empty,Russia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300070006,2026-03-07,false,empty
10394,zcrm_3584690000526813001,2025-03-07 09:15:43,Валерия,Луц,koroleera@mail.ru,empty,+6285190238504,empty,empty,Indonesia,...,Karma Beach,Web Enquiry,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300070003,2026-03-07,false,empty
10395,zcrm_3584690000526819001,2025-03-07 10:23:57,Geoffrey,Aldigeri,geof38@hotmail.com,empty,+6287814992271,empty,empty,Indonesia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300070004,2026-03-07,false,empty


In [4]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04eXpvS1NXTWF2RVE\_BOOKING\Booking By Year\FlyBuy Booking Data (BGAU & BGID & KE).xlsx'
df_booking = pd.read_excel(path, dtype='string', sheet_name='BOOKING_BGAU_BGID_KE')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


# ====================== Awal Filter Umum ====================== 

In [5]:
# data phone, mobile dam email member yang ada di master data (wash member)

m.clean_number(zoho_df, 'Phone')
m.clean_number(zoho_df, 'Mobile')
m.replace_not_valid_to_empty(zoho_df, 'Phone')
m.replace_not_valid_to_empty(zoho_df, 'Mobile')
zoho_df['Email'] = m.lowercase(zoho_df, 'Email')
m.clean_space(zoho_df, 'Email')
m.clean_not_valid_email_format(zoho_df, 'Email')

m.clean_number(df_booking, 'Phone1')
m.clean_number(df_booking, 'Mobile')
df_booking['Email'] = m.lowercase(df_booking, 'Email')
m.clean_space(df_booking, 'Email')
m.clean_not_valid_email_format(df_booking, 'Email')

m.clean_number(df_phone, 'Mobile')

df_email['Email'] = m.lowercase(df_email, 'Email')
m.clean_space(df_email, 'Email')
m.clean_not_valid_email_format(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\main_module.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)
c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\main_module.py:72: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setti

'Email'

In [6]:
count_deleted_wrong_format = m.delete_email_wrong_format(zoho_df, 'Email')
count_after_deleted_wrong_format = zoho_df.shape[0]

# ====================== Akhir Filter Umum ====================== 

In [7]:
# hitung jumlah duplicate
count_duplicate_general = zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])].shape[0]

zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])]

,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,...,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?,Mobile
975,zcrm_3584690000206499154,2020-07-22 08:24:35,Thierry,Nolibr,thierry1970@yahoo.com,empty,6282147470330,empty,empty,Indonesia,...,Karma Beach,Web Enquiry,Karma Beach Bali offer sign up,empty,empty,http://karmagroup.com/karma-beach/sign-up,empty,empty,false,empty
1099,zcrm_3584690000243851842,2020-11-09 11:38:47,Qaul,Siregar,qaulsiregar@gmail.com,1997-09-06,6282166526245,empty,empty,Indonesia,...,Karma Beach,Karma Beach VIP,Karma Beach VIP membership sign up,General,empty,https://karmagroup.com/karma-beach-vip/,2020001100094227,2021-11-09,false,empty
1128,zcrm_3584690000243878881,2020-11-09 12:44:32,Simone,Tait,simotait@gmail.com,1985-07-14,628118306525,empty,empty,Indonesia,...,Karma Beach,Karma Beach VIP,Karma Beach VIP membership sign up,General,empty,https://karmagroup.com/karma-beach-vip/,2020001100092965,2021-11-09,false,empty
1140,zcrm_3584690000243888528,2020-11-09 13:43:10,Salvatore,Adaldo,salvatoreadaldosg@gmail.com,1990-01-15,6594499542,empty,empty,Singapore,...,Karma Beach,Karma Beach VIP,Karma Beach VIP membership sign up,General,empty,https://karmagroup.com/karma-beach-vip/,2020001100092751,2021-11-09,false,empty
1190,zcrm_3584690000243901901,2020-11-09 15:50:44,Ari Widya,Merta,are.rick@ymail.com,1989-03-22,6282147034957,empty,empty,Indonesia,...,Karma Beach,Karma Beach VIP,Karma Beach VIP membership sign up,General,empty,https://karmagroup.com/karma-beach-vip/,2020001100094908,2021-11-09,true,empty
1195,zcrm_3584690000243902866,2020-11-09 15:36:48,Fouad,Hammond,annelinefouad@hotmail.com,1971-02-19,61434242684,empty,empty,Australia,...,Karma Beach,Karma Beach VIP,Karma Beach VIP membership sign up,General,empty,https://karmagroup.com/karma-beach-vip/,2020001100093122,2021-11-09,false,empty
1203,zcrm_3584690000243918799,2020-11-09 18:18:45,Linawaty,Makmur,l_makmur@yahoo.com,1970-04-15,628161831138,empty,empty,Indonesia,...,Karma Beach,Karma Beach VIP,Karma Beach VIP membership sign up,General,empty,https://karmagroup.com/karma-beach-vip/,2020001100098707,2021-11-09,false,empty
1276,zcrm_3584690000244377735,2020-11-10 14:17:07,Wendy,Angelina,wendy_angelina93@hotmail.com,1993-11-12,6281319590449,empty,empty,Indonesia,...,Karma Beach,Karma Beach VIP,Karma Beach VIP membership sign up,General,empty,https://karmagroup.com/karma-beach-vip/,2020001100100158,2021-11-10,false,empty
1293,zcrm_3584690000244393749,2020-11-10 12:36:12,Alexander,Quinders,funnyfarm@free.fr,1961-03-01,6287860586536,empty,empty,Indonesia,...,Karma Beach,Karma Beach VIP,Karma Beach VIP membership sign up,General,empty,https://karmagroup.com/karma-beach-vip/,2020001100102565,2021-11-10,true,empty
1360,zcrm_3584690000244667249,2020-11-12 02:35:47,Alison,Malkin,ali.malkin78@gmail.com,1978-07-24,61414579145,empty,empty,Australia,...,Karma Beach,Karma Beach VIP,Karma Beach VIP membership sign up,General,empty,https://karmagroup.com/karma-beach-vip/,2020001100128985,2021-11-12,false,empty


In [8]:
zoho_df.drop_duplicates(subset=['Phone','Mobile','Email'], inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = zoho_df.shape[0]

zoho_df

,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,...,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?,Mobile
0,zcrm_3584690000001379021,2019-02-03 08:42:45,Keli,Parkes,keli@keliaroha.com,1976-05-14,61413422227,"17 Fishnourne rd, Allambie heights",Sydney,Australia,...,Other,Web Enquiry,Karma Beach VIP membership sign up 2023,empty,"Other - Web Enquiry, Karma Club - Membership,...",https://karmagroup.com/karma-beach-vip/,2024000100260004,2025-01-26,false,empty
1,zcrm_3584690000002932135,2019-02-28 03:24:54,Almas Hussain,Rizvi,almas_h@yahoo.com,empty,919886790473,empty,empty,India,...,Other,Facebook Ads,Karma Beach VIP membership sign up 2023,General,Instagram,https://karmagroup.com/karma-beach-vip/,2024000100090124,2025-01-09,false,empty
2,zcrm_3584690000005944396,2019-04-02 02:14:44,Caryl,Costello,carylcostello@yahoo.com.au,1940-01-01,61417074510,"131 College Road, Karana Downs",Brisbane,Australia,...,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Jimbaran,"Other - Past Guests, Other - Web Chat, Karma C...",https://karmagroup.com/karma-beach-vip/,2024000800300003,2025-08-30,false,empty
3,zcrm_3584690000005945085,2019-04-02 02:14:48,Dawn,Poole,chrisdawnpoole@gmail.com,empty,61417854221,empty,empty,Australia,...,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Bavaria,"Other - Past Guests, Other - Other",https://karmagroup.com/karma-beach-vip/,2024000500230006,2025-05-23,false,empty
4,zcrm_3584690000005948267,2019-04-02 02:15:06,Elycia,Martins,elycia.martins@gmail.com,empty,61403924069,empty,empty,Australia,...,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Rottnest,Other - Past Guests,https://karmagroup.com/karma-beach-vip/,2024000900090001,2025-09-09,false,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10346,zcrm_3584690000526811033,2025-03-07 09:13:36,Ekaterina,Terekhina,katyess@icloud.com,empty,79136194604,empty,empty,Russia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300070002,2026-03-07,false,empty
10347,zcrm_3584690000526812042,2025-03-07 10:49:34,Juliia,Parshenkova,ylashka_1@mail.ru,empty,79060677484,empty,empty,Russia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300070006,2026-03-07,false,empty
10348,zcrm_3584690000526813001,2025-03-07 09:15:43,Валерия,Луц,koroleera@mail.ru,empty,6285190238504,empty,empty,Indonesia,...,Karma Beach,Web Enquiry,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300070003,2026-03-07,false,empty
10349,zcrm_3584690000526819001,2025-03-07 10:23:57,Geoffrey,Aldigeri,geof38@hotmail.com,empty,6287814992271,empty,empty,Indonesia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300070004,2026-03-07,false,empty


In [9]:
# Cek leads test
test_leads = zoho_df.loc[
    
        zoho_df['Email'].str.contains('^test@', flags=re.I, regex=True) |
        zoho_df['Email'].str.contains('\+test', flags=re.I, regex=True) |
    
        zoho_df['First Name'].str.contains('^test$', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('^test$', flags=re.I, regex=True) |

        zoho_df['First Name'].str.contains('test 1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test 2', flags=re.I, regex=True)|
        zoho_df['First Name'].str.contains('test1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test2', flags=re.I, regex=True)|

        zoho_df['First Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test$', flags=re.I, regex=True)
    ]

count_leads_test = test_leads.shape[0]

# test_leads.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\test_leads_2.xlsx', index=False)
test_leads

,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,...,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?,Mobile


In [10]:
zoho_df.drop(test_leads.index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_dedup_test = zoho_df.shape[0]

In [11]:
# dnm_leads = zoho_df.loc[(zoho_df['Do Not Mail'] == 'true') | (zoho_df['Do Not Contact'] == 'true')]
# count_dnm = dnm_leads.shape[0]

In [12]:
# zoho_df.drop(dnm_leads.index, inplace=True)
# zoho_df.reset_index(drop=True, inplace=True)

# count_after_dedup_dnm = zoho_df.shape[0]

In [13]:
# Cek Email OTA
email_OTA = m.find_OTA(zoho_df, 'Email')
count_email_OTA = email_OTA.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\email_OTA.xlsx', index=False)

email_OTA

,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,...,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?,Mobile
6543,zcrm_3584690000491845127,2024-04-14 00:26:53,Ankit,Agarwal,ankit@yotours.in,empty,917276282020,empty,empty,India,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2024000400140001,2025-04-14,false,empty
6544,zcrm_3584690000491845156,2024-04-14 00:29:07,Priyanka,Mangal,info@yotours.in,empty,918451823654,empty,empty,India,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2024000400140002,2025-04-14,false,empty
8447,zcrm_3584690000512153086,2024-10-17 13:15:08,Tamara,Skroman,supermarkettours@mail.ru,empty,6281225297099,empty,empty,Indonesia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2024001000170008,2025-10-17,false,empty
10137,zcrm_3584690000525549244,2025-02-25 18:07:41,Hayley,White,scootontour@hotmail.com,empty,61415187714,empty,empty,Australia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000200250011,2026-02-25,false,empty


In [14]:
# delete email OTA
count_after_clean_OTA = m.delete_OTA(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,...,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?,Mobile
0,zcrm_3584690000001379021,2019-02-03 08:42:45,Keli,Parkes,keli@keliaroha.com,1976-05-14,61413422227,"17 Fishnourne rd, Allambie heights",Sydney,Australia,...,Other,Web Enquiry,Karma Beach VIP membership sign up 2023,empty,"Other - Web Enquiry, Karma Club - Membership,...",https://karmagroup.com/karma-beach-vip/,2024000100260004,2025-01-26,false,empty
1,zcrm_3584690000002932135,2019-02-28 03:24:54,Almas Hussain,Rizvi,almas_h@yahoo.com,empty,919886790473,empty,empty,India,...,Other,Facebook Ads,Karma Beach VIP membership sign up 2023,General,Instagram,https://karmagroup.com/karma-beach-vip/,2024000100090124,2025-01-09,false,empty
2,zcrm_3584690000005944396,2019-04-02 02:14:44,Caryl,Costello,carylcostello@yahoo.com.au,1940-01-01,61417074510,"131 College Road, Karana Downs",Brisbane,Australia,...,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Jimbaran,"Other - Past Guests, Other - Web Chat, Karma C...",https://karmagroup.com/karma-beach-vip/,2024000800300003,2025-08-30,false,empty
3,zcrm_3584690000005945085,2019-04-02 02:14:48,Dawn,Poole,chrisdawnpoole@gmail.com,empty,61417854221,empty,empty,Australia,...,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Bavaria,"Other - Past Guests, Other - Other",https://karmagroup.com/karma-beach-vip/,2024000500230006,2025-05-23,false,empty
4,zcrm_3584690000005948267,2019-04-02 02:15:06,Elycia,Martins,elycia.martins@gmail.com,empty,61403924069,empty,empty,Australia,...,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Rottnest,Other - Past Guests,https://karmagroup.com/karma-beach-vip/,2024000900090001,2025-09-09,false,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10342,zcrm_3584690000526811033,2025-03-07 09:13:36,Ekaterina,Terekhina,katyess@icloud.com,empty,79136194604,empty,empty,Russia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300070002,2026-03-07,false,empty
10343,zcrm_3584690000526812042,2025-03-07 10:49:34,Juliia,Parshenkova,ylashka_1@mail.ru,empty,79060677484,empty,empty,Russia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300070006,2026-03-07,false,empty
10344,zcrm_3584690000526813001,2025-03-07 09:15:43,Валерия,Луц,koroleera@mail.ru,empty,6285190238504,empty,empty,Indonesia,...,Karma Beach,Web Enquiry,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300070003,2026-03-07,false,empty
10345,zcrm_3584690000526819001,2025-03-07 10:23:57,Geoffrey,Aldigeri,geof38@hotmail.com,empty,6287814992271,empty,empty,Indonesia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300070004,2026-03-07,false,empty


In [15]:
# Cek Email Karma
email_karma = m.find_karma(zoho_df, 'Email')
count_email_karma = email_karma.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\email_OTA.xlsx', index=False)

email_karma

,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,...,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?,Mobile
5342,zcrm_3584690000472907352,2023-12-14 08:24:49,Shaun,Duenhofen,shaun.duenhofen@karmakandara.com,empty,6281138006892,empty,empty,Indonesia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2023001200140001,2024-12-14,false,empty
5343,zcrm_3584690000472907381,2023-12-14 09:12:09,James,Casey,james.casey@karmagroup.com,empty,628123601872,empty,empty,Indonesia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2023001200140004,2024-12-14,false,empty
5723,zcrm_3584690000476213047,2024-01-10 06:59:19,Michael,Wood,michael.wood@karmagroup.com,empty,628111111111,empty,empty,Indonesia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2024000100100020,2025-01-10,false,empty
9552,zcrm_3584690000522488067,2025-01-27 10:24:16,Arditra,Test Booking,resmgr@karmaresorts.com,empty,623618482200,empty,empty,Indonesia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000100270003,2026-01-27,false,empty


In [16]:
# delete email Karma
count_after_clean_karma = m.delete_karma(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,...,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?,Mobile
0,zcrm_3584690000001379021,2019-02-03 08:42:45,Keli,Parkes,keli@keliaroha.com,1976-05-14,61413422227,"17 Fishnourne rd, Allambie heights",Sydney,Australia,...,Other,Web Enquiry,Karma Beach VIP membership sign up 2023,empty,"Other - Web Enquiry, Karma Club - Membership,...",https://karmagroup.com/karma-beach-vip/,2024000100260004,2025-01-26,false,empty
1,zcrm_3584690000002932135,2019-02-28 03:24:54,Almas Hussain,Rizvi,almas_h@yahoo.com,empty,919886790473,empty,empty,India,...,Other,Facebook Ads,Karma Beach VIP membership sign up 2023,General,Instagram,https://karmagroup.com/karma-beach-vip/,2024000100090124,2025-01-09,false,empty
2,zcrm_3584690000005944396,2019-04-02 02:14:44,Caryl,Costello,carylcostello@yahoo.com.au,1940-01-01,61417074510,"131 College Road, Karana Downs",Brisbane,Australia,...,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Jimbaran,"Other - Past Guests, Other - Web Chat, Karma C...",https://karmagroup.com/karma-beach-vip/,2024000800300003,2025-08-30,false,empty
3,zcrm_3584690000005945085,2019-04-02 02:14:48,Dawn,Poole,chrisdawnpoole@gmail.com,empty,61417854221,empty,empty,Australia,...,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Bavaria,"Other - Past Guests, Other - Other",https://karmagroup.com/karma-beach-vip/,2024000500230006,2025-05-23,false,empty
4,zcrm_3584690000005948267,2019-04-02 02:15:06,Elycia,Martins,elycia.martins@gmail.com,empty,61403924069,empty,empty,Australia,...,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Rottnest,Other - Past Guests,https://karmagroup.com/karma-beach-vip/,2024000900090001,2025-09-09,false,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10338,zcrm_3584690000526811033,2025-03-07 09:13:36,Ekaterina,Terekhina,katyess@icloud.com,empty,79136194604,empty,empty,Russia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300070002,2026-03-07,false,empty
10339,zcrm_3584690000526812042,2025-03-07 10:49:34,Juliia,Parshenkova,ylashka_1@mail.ru,empty,79060677484,empty,empty,Russia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300070006,2026-03-07,false,empty
10340,zcrm_3584690000526813001,2025-03-07 09:15:43,Валерия,Луц,koroleera@mail.ru,empty,6285190238504,empty,empty,Indonesia,...,Karma Beach,Web Enquiry,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300070003,2026-03-07,false,empty
10341,zcrm_3584690000526819001,2025-03-07 10:23:57,Geoffrey,Aldigeri,geof38@hotmail.com,empty,6287814992271,empty,empty,Indonesia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300070004,2026-03-07,false,empty


In [17]:
find_member_booking = zoho_df.copy()

find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1'
find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Email'].isin(df_booking['Email']), 'Same Value Find'] = 'Booking-Email'

find_member_booking.loc[find_member_booking['Phone'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'

find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

count_booking_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Booking', flags=re.I, regex=True)].shape[0]
count_member_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Member', flags=re.I, regex=True)].shape[0]

find_member_booking.replace('empty', '', inplace=True)
# find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\booking-member-dedup.csv', index=False)
# find_member_booking.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\booking-member-dedup.xlsx', index=False)

#find_member_booking.loc[find_member_booking['Same Value Find'] == 'Booking-Phone1'] 

In [18]:
# data phone, mobile dam email member yang ada di master data 
count_wash_member = zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email'])| 

    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

].shape[0]

zoho_df.drop(zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email']) |
    
    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

    ].index, inplace=True)
    
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_member = zoho_df.shape[0]


In [19]:
count_email_only = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] == 'empty'))].shape[0]
count_phone_only = zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty'))].shape[0]
count_email_and_phone = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty'))].shape[0]

count_blank = zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].shape[0]

# zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\blank_data.xlsx', index=False)

zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] == 'empty')), 'Contact Type'] = 'Email Only'
zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Phone Only'
zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Email and Phone'

zoho_df.drop(zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_blank = zoho_df.shape[0]

In [20]:
arr_dedup = [count_deleted_wrong_format, count_duplicate_general, count_leads_test, count_email_OTA, count_email_karma, count_blank]
count_sum_dedup = sum(arr_dedup)
count_sum_dedup

54

In [21]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Filter Leads Awal : "+str(count_filter_awal))
print("Jumlah Format Email Salah : "+str(count_deleted_wrong_format))
print("Jumlah Setelah Clean Format Email Salah : "+str(count_after_deleted_wrong_format))
print("Jumlah Duplicate (Phone, Mobile dan Email) : "+str(count_duplicate_general))
print("Jumlah Setelah Clean Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Lead Test : "+str(count_leads_test))
print("Jumlah Setelah Clean Lead Test : "+str(count_after_dedup_test))
# print("Jumlah Lead Do Not Mail : "+str(count_dnm))
# print("Jumlah Setelah Clean Lead Do Not Mail : "+str(count_after_dedup_dnm))
print("Jumlah Email OTA : "+str(count_email_OTA))
print("Jumlah Setelah Clean Email OTA : "+str(count_after_clean_OTA))
print("Jumlah Email Karma : "+str(count_email_karma))
print("Jumlah Setelah Clean Email Karma : "+str(count_after_clean_karma))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Setelah Wash Member : "+str(count_after_wash_member))
print("Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : "+str(count_blank))
print("Jumlah Setelah Wash Phone Email Blank : "+str(count_after_wash_blank))
print("==========================================================")
print("Contact Type")
print("==========================================================")
print("Jumlah Email Only : "+str(count_email_only))
print("Jumlah Phone Only : "+str(count_phone_only))
print("Jumlah Email dan Phone : "+str(count_email_and_phone))

Summary
Jumlah Filter Leads Awal : 10397
Jumlah Format Email Salah : 0
Jumlah Setelah Clean Format Email Salah : 10397
Jumlah Duplicate (Phone, Mobile dan Email) : 46
Jumlah Setelah Clean Duplicate : 10351
Jumlah Lead Test : 0
Jumlah Setelah Clean Lead Test : 10351
Jumlah Email OTA : 4
Jumlah Setelah Clean Email OTA : 10347
Jumlah Email Karma : 4
Jumlah Setelah Clean Email Karma : 10343
Jumlah Wash Member : 832
Jumlah Wash Booking : 108
Jumlah Setelah Wash Member : 9403
Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : 0
Jumlah Setelah Wash Phone Email Blank : 9403
Contact Type
Jumlah Email Only : 8
Jumlah Phone Only : 197
Jumlah Email dan Phone : 9198


In [22]:
print("==========================================================")
print("Summary Report")
print("==========================================================")
print("Jumlah Data Awal : "+str(count_filter_awal))
print("Jumlah Wash Dedup : "+str(count_sum_dedup))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Data Akhir : "+str(count_after_wash_blank))

Summary Report
Jumlah Data Awal : 10397
Jumlah Wash Dedup : 54
Jumlah Wash Member : 832
Jumlah Wash Booking : 108
Jumlah Data Akhir : 9403


In [23]:
zoho_df.replace('empty', '', inplace=True, regex=True)

zoho_df['First Name'] = zoho_df['First Name'].str.title()
zoho_df['Last Name'] = zoho_df['Last Name'].str.title()

# zoho_df.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\karma beach cleand.csv', index=False)
zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\past guest kb german speaking clean.xlsx', index=False)